# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `Salary.sex` の `'m'` と `'f'` を **単一操作**で相互に入れ替える
* 入力 DF: `Salary(id: int, name: str, sex: {'m','f'}, salary: int)`
* 出力: 列名・順序そのまま（`['id','name','sex','salary']`）、`sex` が `'m' <-> 'f'` に反転。他列は不変

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → ベクトル置換（`map`）→ 条件ガード（`isin`）で安全に反転。`m/f` 以外（将来の拡張や `NaN`）は温存。

```python
import pandas as pd

def swap_sex(Salary: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'name', 'sex', 'salary']
    """
    # 列最小化（順序保持）
    out = Salary[["id", "name", "sex", "salary"]].copy()

    # m/f のみを対象にトグル。その他の値（存在すれば）はそのまま温存
    mask = out["sex"].isin(["m", "f"])
    out.loc[mask, "sex"] = out.loc[mask, "sex"].map({"m": "f", "f": "m"}).values

    return out

Analyze Complexity
Runtime 267 ms
Beats 36.94%
Memory 66.92 MB
Beats 7.25%

```

* ポイント: `where` でも同等に書けます（可読性はお好みで）。

  ```python
  out["sex"] = out["sex"].where(~mask, out["sex"].map({"m": "f", "f": "m"}))
  ```

## 3) アルゴリズム説明

* 使用 API: `DataFrame.__getitem__`（列選択）, `copy`, `Series.isin`, `Series.map`, `DataFrame.loc`
* **NULL / 重複 / 型**:

  * `map` は辞書外の値を `NaN` にしがち → 事前に `isin` で対象を限定して **非対象は温存**。
  * 行の重複は本問題では無関係（全行独立変換）。
  * 列型は `object`/`string[pyarrow]` いずれでも動作。`Categorical` の場合はカテゴリに `'m','f'` がある前提。

## 4) 計算量（概算）

* 全体 **O(N)**、追加メモリは `sex` 列相当の一時 Series（マスク＋置換分）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム Salary] --> B[列最小化 id name sex salary]
  B --> C["mask で sex in {m f} 抽出"]
  C --> D[map で m↔f 置換 非対象は温存]
  D --> E[出力 仕様列のみ]
```

# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `Salary.sex` の `'m'` と `'f'` を **単一操作**で相互に入れ替える
* 入力 DF: `Salary(id: int, name: str, sex: {'m','f'}, salary: int)`
* 出力: 列名・順序そのまま（`['id','name','sex','salary']`）、`sex` が `'m' <-> 'f'` に反転。他列は不変

## 2) 実装（指定シグネチャ厳守）

> **メモリ削減＆高速化**：`NumPy` 配列に直アクセスし、対象行のみを一括更新。
> 文字列列は `object`/`string` を想定。`Categorical` の場合は **コード反転**が最速です。

```python
import pandas as pd
import numpy as np

def swap_sex(Salary: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'name', 'sex', 'salary']
    """
    cols = ["id", "name", "sex", "salary"]
    # DataFrame 全体の copy を避け、必要列だけを参照（CoW で不要コピーを抑制）
    base = Salary[cols]

    s = base["sex"]

    # 1) Categorical の場合はコード入替が最小メモリ・最速
    if pd.api.types.is_categorical_dtype(s):
        codes = s.cat.codes.to_numpy(copy=False)  # -1 は NaN
        mask = codes >= 0                        # 実データのみ
        # 'm','f' の2値想定なので 0/1 を反転
        new_codes = codes.copy()
        new_codes[mask] = 1 - new_codes[mask]
        new_sex = pd.Categorical.from_codes(
            new_codes, categories=s.cat.categories, ordered=s.cat.ordered
        )
        # assign で列差分だけ差し替え（他列はブロック共有でメモリ圧縮）
        return base.assign(sex=new_sex)

    # 2) 文字列/オブジェクト列：配列ビューで一括置換（辞書 map より中間オブジェクトが少ない）
    arr = s.to_numpy(copy=False)                # 参照（書込可能でない場合もあるため後で安全に再代入）
    mask = (arr == "m") | (arr == "f")          # 対象行だけ
    if mask.any():
        tmp = arr[mask]
        swapped = np.where(tmp == "m", "f", "m")
        # 再代入で dtype を保ちながら差し替え
        return base.assign(sex=pd.Series(arr, index=s.index).where(~mask, swapped))
    else:
        # m/f が無いならそのまま返却
        return base

Analyze Complexity
Runtime 246 ms
Beats 79.21%
Memory 66.92 MB
Beats 7.25%

```

### 追加の実務メモ（任意）

* `sex` を **Categorical**（例: `pd.CategoricalDtype(['m','f'])`）にしておくと、**メモリ削減**かつ本トグルが最小コストになります。
* `string[pyarrow]` でも良いですが、2 値なら `Categorical` がより省メモリ。

## 3) アルゴリズム説明

* 使用 API:

  * `DataFrame.__getitem__` で **列最小化**
  * `Series.to_numpy(copy=False)` で **ゼロコピー参照**（必要時のみ再割当）
  * `numpy.where` による **ベクトル条件置換**
  * `Series.cat.codes` / `Categorical.from_codes` による **カテゴリコード反転**
  * `DataFrame.assign` で **差分列のみ差し替え**（他列ブロックは共有されやすく、メモリ節約）
* **NULL / 異常値**:

  * `Categorical` の `codes == -1`（= NaN）は非対象として温存。
  * 文字列系でも `mask` 外は温存（`NaN` や `'u'` などが混ざっても壊さない）。

## 4) 計算量（概算）

* 時間: 全体 **O(N)**（マスク作成＋部分置換）
* 追加メモリ: **O(K)**（`K` は `sex ∈ {'m','f'}` の行数）

  * `map` 方式より一時 Series が小さく、**ピークメモリ削減**が見込めます。
  * `Categorical` の場合は **整数配列のみ**を一時利用。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 DF Salary] --> B[列最小化 id name sex salary]
  B --> C[sex が Categorical なら codes を反転]
  B --> D[文字列なら mask を作成し where で部分置換]
  C --> E[assign で sex 差替え]
  D --> E[assign で sex 差替え]
  E --> F[出力 列順は id name sex salary]
```

---

### さらなる微調整（ベンチ改善の現実解）

* **前処理で `sex` を Categorical にする**：

  ```python
  Salary["sex"] = pd.Categorical(Salary["sex"], categories=["m","f"])
  ```

  以後のトグルは **整数反転のみ**になり、**Runtime と Memory の双方が安定して改善**します。
* **巨大 DF** ならバッチ処理（例: `np.array_split` でインデックス分割→ `concat`）でピークメモリを抑制可能（総時間は微増）。
* **JIT** は不要。I/O も禁止条件のため、**今回の改良余地は配列直操作と Categorical 化**が最も効きます。
